<a href="https://colab.research.google.com/github/soadD/Data-Science-Projects/blob/master/Box_and_one_dinasty_ED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Projeto com uma liga de Fantasy NBA : PARTE 2 - ENRIQUECIMENTO DE DATASET (ED)

Este projeto tem como fim tirar as principais análises dos times que foram aos playoffs (12) de uma liga de Fantasy Basketball formato dinastia. A partir deste serão coletadas métricas relevantes de como montar um time competitivo com chances de ganhar um título.

Dados:

Informações sobre times, colocações e rosters:

**MadHatter Fantasy Leagues**

 https://discord.gg/qXGxSbws

Estatísticas sobre jogadores, liga e tudo relacionado a basquete:

**nba_api**

https://github.com/swar/nba_api/tree/master?tab=readme-ov-file

**NBA Stats**

https://www.nba.com/stats


## Extração dos dados

### Importando times e seus jogadores que foram aos playoffs na liga Box-and-one Dinasty League

In [ ]:
#Comando de abertura para google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importação de bibliotecas
import pandas as pd

In [ ]:
# Extração do csv da liga Box-and-one Dinasty
box = pd.read_csv('/content/drive/MyDrive/NBA/ETL/box_and_one_dinasty.csv')

### Importando tabela com todos os jogadores da NBA temporada 23-24

In [ ]:
#Importando bibliotecas
import requests

In [ ]:
#URL com estatísticas dos jogadores da NBA para a temporada regular de 23-24
url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=2023-24&SeasonType=Regular%20Season&StatCategory=PTS'

In [ ]:
#Importação dos arquivos dessa url para uma variável no ambiente virtual
r = requests.get(url=url).json()

In [ ]:
#Como no JSON a descrição das estatísticas de cada jogador está armazenado em headers, vamos chamá-la como topo de tabela
table_headers = r['resultSet']['headers']

In [ ]:
#Criação de dataframe com valores do json e nomeando as colunas com table_headers
players = pd.DataFrame(r['resultSet']['rowSet'], columns=table_headers)

## Transformação

### Tipos de dados

In [ ]:
# Verificação dos tipos de dados
box.dtypes

PLAYER_ID      int64
RANK           int64
PLAYER        object
TEAM_ID        int64
TEAM          object
GP             int64
MIN            int64
FGM            int64
FGA            int64
FG_PCT       float64
FG3M           int64
FG3A           int64
FG3_PCT      float64
FTM            int64
FTA            int64
FT_PCT       float64
OREB           int64
DREB           int64
REB            int64
AST            int64
STL            int64
BLK            int64
TOV            int64
PF             int64
PTS            int64
EFF            int64
AST_TOV      float64
STL_TOV      float64
team          object
dtype: object

In [ ]:
# Transformar PLAYER_ID e TEAM_ID em object, renomear team para OWNER
# Convertendo o tipo de dados numéricos de float para int
box['PLAYER_ID'] = box['PLAYER_ID'].astype(str)
box['TEAM_ID'] = box['TEAM_ID'].astype(str)
box.rename(columns={'team': 'OWNER'}, inplace=True)

### Verificação de valores nulos


In [ ]:
# Verificando valores nulos
box.isnull().sum()

PLAYER_ID    0
RANK         0
PLAYER       0
TEAM_ID      0
TEAM         0
GP           0
MIN          0
FGM          0
FGA          0
FG_PCT       0
FG3M         0
FG3A         0
FG3_PCT      0
FTM          0
FTA          0
FT_PCT       0
OREB         0
DREB         0
REB          0
AST          0
STL          0
BLK          0
TOV          0
PF           0
PTS          0
EFF          0
AST_TOV      0
STL_TOV      0
OWNER        0
dtype: int64

## Enriquecimento de Dataset BOX AND ONE


## Enriquecimento de Dataset PLAYERS


### Adicionando Z-Scores do Basketball Monster (BBM)

In [ ]:
#Importação de Z-Scores
zscores = pd.read_csv('/content/drive/MyDrive/NBA/Zscores/zscores.csv')

In [ ]:
#Transformar zscores em dataframe type
zscores = pd.DataFrame(zscores)

In [ ]:
#Visualização do dataframe
zscores.head(1)

,Round,Rank,Value,Name,Team,Pos,Inj,g,m/g,p/g,...,2a/g,pV,3V,rV,aV,sV,bV,fg%V,ft%V,toV
0,1,1,"1,060554395",Joel Embiid,PHI,C,NaN,39,"33,57393162","34,69230769",...,"18,25641026","2,937799812","-0,3285533786","2,022542343","0,7595001018","0,5958714268","1,776745735","1,248958118","3,044786661","-2,512661268"


In [ ]:
#Renomear "value" para "zscore"
zscores.rename(columns={'Value': 'Zscore'}, inplace=True)
zscores.rename(columns={'Rank': 'BBM_RANK'}, inplace=True)

In [ ]:
# Selecionar apenas a coluna 'zscore' do dataframe zscores para dar merge com box_pg
zscores_subset = zscores[['Name', 'Zscore', 'BBM_RANK']]

In [ ]:
#Verificando tipos em zscores_subset
zscores_subset.dtypes

Name        object
Zscore      object
BBM_RANK     int64
dtype: object

In [ ]:
#Transformando Zscore para float convertendo virgulas para pontos
zscores_subset['Zscore'] = zscores_subset['Zscore'].str.replace(',', '.').astype(float)

<ipython-input-195-a73eebfdb44e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zscores_subset['Zscore'] = zscores_subset['Zscore'].str.replace(',', '.').astype(float)


In [ ]:
#zscore com apenas 2 casas decimais
zscores_subset['Zscore'] = zscores_subset['Zscore'].round(2)

<ipython-input-196-68caaeee304e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zscores_subset['Zscore'] = zscores_subset['Zscore'].round(2)


In [ ]:
#Assegurar que tanto zscores_subset quanto box_pg são DataFrames do pandas
zscores_subset = pd.DataFrame(zscores_subset)

In [ ]:
#Deixando chave principal e chave estrangeira com mesmo nome
zscores_subset.rename(columns={'Name': 'PLAYER'}, inplace=True)

In [ ]:
# Realizar o merge usando a coluna "PLAYER" como chave
box_df = pd.merge(box, zscores_subset, on='PLAYER', how='left')

In [ ]:
box_df

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV,OWNER,Zscore,BBM_RANK
0,1630163,210,LaMelo Ball,1610612766,CHA,22,711,183,423,0.433,...,4,83,79,526,522,2.12,0.48,Buzz,0.29,23.0
1,1630217,97,Desmond Bane,1610612763,MEM,42,1443,360,776,0.464,...,22,112,120,997,928,2.05,0.38,Buzz,0.19,33.0
2,1626179,64,Terry Rozier,1610612748,MIA,61,2040,442,998,0.443,...,21,106,101,1205,1190,3.23,0.59,Buzz,0.05,54.0
3,1630530,131,Trey Murphy III,1610612740,NOP,57,1690,275,621,0.443,...,28,34,70,842,917,3.65,1.50,Buzz,NaN,NaN
4,1629023,106,P.J. Washington,1610612742,DAL,73,2221,354,811,0.436,...,59,96,169,939,1023,1.47,0.75,Buzz,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1628998,337,Cody Martin,1610612766,CHA,28,753,82,215,0.381,...,18,44,48,211,284,2.36,0.73,31savage,NaN,NaN
210,1629007,397,Jontay Porter,1610612761,TOR,26,360,40,104,0.385,...,20,21,35,115,209,2.86,0.95,31savage,NaN,NaN
211,1630568,406,Luka Garza,1610612750,MIN,25,122,36,75,0.480,...,1,6,24,99,87,0.67,0.67,31savage,NaN,NaN
212,1630551,414,Justin Champagnie,1610612764,WAS,15,235,32,78,0.410,...,9,10,14,89,121,1.90,1.00,31savage,NaN,NaN


In [ ]:
unmatched_zscores = zscores_subset[~zscores_subset['PLAYER'].isin(box['PLAYER'])]
unmatched_box = box[~box['PLAYER'].isin(zscores_subset['PLAYER'])]

print("Unmatched in zscores_subset:", unmatched_zscores)
print("Unmatched in box:", unmatched_box)

Unmatched in zscores_subset:                       PLAYER  Zscore  BBM_RANK
0                Joel Embiid    1.06         1
1    Shai Gilgeous-Alexander    0.99         2
9          Tyrese Haliburton    0.48        10
13        Kristaps Porzingis    0.41        14
18              Tyrese Maxey    0.30        19
..                       ...     ...       ...
181       Brandin Podziemski   -0.39       182
183         Taylor Hendricks   -0.40       184
184           Brandon Clarke   -0.40       185
185           Shaedon Sharpe   -0.40       186
187            Cason Wallace   -0.41       188

[91 rows x 3 columns]
Unmatched in box:     PLAYER_ID  RANK             PLAYER     TEAM_ID TEAM  GP   MIN  FGM  FGA  \
3     1630530   131    Trey Murphy III  1610612740  NOP  57  1690  275  621   
4     1629023   106    P.J. Washington  1610612742  DAL  73  2221  354  811   
7     1630631   250      Jose Alvarado  1610612740  NOP  56  1028  142  345   
8     1630214   298     Xavier Tillman  1610612738

### Adicionando FBI Dinasty Ranks para Box_df

In [ ]:
#Importação de tabela
fbi = pd.read_csv('/content/drive/MyDrive/NBA/FBI Dinasty Rankings/FBI_dinasty_rankings_fixed.csv', sep = ';')

In [ ]:
fbi

,Rank,Unnamed: 1,Player,Team,Unnamed: 4,Birthdate,Pos.,Age,Change,Projected Peak Stat Set,Stat Set Score
0,1,NaN,Victor Wembanyama,SAS,NaN,01/04/2004,C,20.5,0,FG% PTS 3PT REB AST STL BLK FT% TO,⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐⭐⭐⭐⭐⭐
1,2,NaN,Luka Doncic,DAL,NaN,2/28/1999,SF,25.4,0,FG% PTS 3PT REB AST STL BLK FT% TO,⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐
2,3,NaN,Shai Gilgeous-Alexander,OKC,NaN,07/12/1998,PG,26.0,0,FG% PTS 3PT REB AST STL BLK FT% TO,⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐
3,4,NaN,Nikola Jokic,DEN,NaN,2/19/1995,C,29.4,0,FG% PTS 3PT REB AST STL BLK FT% TO,⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐️⭐⭐⭐
4,5,NaN,Tyrese Haliburton,IND,NaN,2/29/2000,PG,24.4,0,FG% PTS 3PT REB AST STL BLK FT% TO,⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐
...,...,...,...,...,...,...,...,...,...,...,...
563,564,NaN,Malcolm Hill,NOR,NaN,10/26/1995,SF,28.7,-50,FG% PTS 3PT REB AST STL BLK FT% TO,NaN
564,565,NaN,James Johnson,IND,NaN,2/20/1987,PF,37.4,-32,Coming Soon,NaN
565,566,NaN,Pete Nance,CLE,NaN,2/19/2000,PF,24.4,-51,Coming Soon,NaN
566,567,NaN,Garrett Temple,TOR,NaN,05/08/1986,SF,38.2,-33,FG% PTS 3PT REB AST STL BLK FT% TO,❌❌❌❌❌❌❌❌❌❌


In [ ]:
#Renomear "value" para "zscore"
fbi.rename(columns={'Player': 'PLAYER'}, inplace=True)
fbi.rename(columns={'Rank': 'FBI_RANK'}, inplace=True)

In [ ]:
# Selecionar apenas a coluna 'zscore' do dataframe zscores para dar merge com box_pg
fbi_subset = fbi[['PLAYER', 'FBI_RANK', 'Age']]

In [ ]:
fbi_subset.dtypes

PLAYER       object
FBI_RANK      int64
Age         float64
dtype: object

In [ ]:
#Falhas no join
unmatched_box = box[~box['PLAYER'].isin(fbi_subset['PLAYER'])]

print("Unmatched in box:", unmatched_box)

Unmatched in box:     PLAYER_ID  RANK             PLAYER     TEAM_ID TEAM  GP  MIN  FGM  FGA  \
33    1630587   399      Isaiah Livers  1610612765  DET  23  469   40  116   
124    202722   294      Davis Bertans  1610612766  CHA  43  674   92  234   
196   1630249   386         Vit Krejci  1610612737  ATL  22  541   51  104   
210   1629007   397      Jontay Porter  1610612761  TOR  26  360   40  104   
212   1630551   414  Justin Champagnie  1610612764  WAS  15  235   32   78   
213   1630610   458      DeJon Jarreau  1610612763  MEM   9  150   17   50   

     FG_PCT  ...  AST  STL  BLK  TOV  PF  PTS  EFF  AST_TOV  STL_TOV     OWNER  
33    0.345  ...   25   13    5   16  43  114  107     1.56     0.81       luk  
124   0.393  ...   35   26   12   18  56  290  261     1.94     1.44       ect  
196   0.490  ...   50   13    6   20  48  135  182     2.50     0.65    hatter  
210   0.385  ...   60   20   20   21  35  115  209     2.86     0.95  31savage  
212   0.410  ...   19   10    

In [ ]:
# Realizar o merge usando a coluna "PLAYER" como chave
box_df = pd.merge(box_df, fbi_subset, on='PLAYER', how='left')

### Adicionando Dizzle Dazzle Ranks para Box_df

In [ ]:
#Importação de tabela
dizzle = pd.read_csv('/content/drive/MyDrive/NBA/Dizzle Dinasty/dizzle_sheets - dizzle_two.csv.csv')

In [ ]:
#Renomear "value" para "zscore"
dizzle.rename(columns={'Player': 'PLAYER'}, inplace=True)
dizzle.rename(columns={'Rank': 'DIZZLE_RANK'}, inplace=True)

In [ ]:
# Selecionar apenas a coluna 'zscore' do dataframe zscores para dar merge com box_pg
dizzle_subset = dizzle[['PLAYER', 'DIZZLE_RANK']]

In [ ]:
dizzle_subset.dtypes

PLAYER         object
DIZZLE_RANK    object
dtype: object

In [ ]:
#Falhas no join
unmatched_box = box_df[~box_df['PLAYER'].isin(dizzle_subset['PLAYER'])]

print("Unmatched in box:", unmatched_box)

Unmatched in box:     PLAYER_ID  RANK             PLAYER     TEAM_ID TEAM  GP  MIN  FGM  FGA  \
33    1630587   399      Isaiah Livers  1610612765  DET  23  469   40  116   
47    1629726   278   Garrison Mathews  1610612737  ATL  66  992  103  226   
52     203933   464        T.J. Warren  1610612750  MIN  11  125   18   41   
68     202684   367   Tristan Thompson  1610612739  CLE  49  549   73  120   
155   1628981   300     Bruno Fernando  1610612737  ATL  45  682  109  187   
176    201599   381     DeAndre Jordan  1610612743  DEN  36  396   58   93   
177   1629674   371      Neemias Queta  1610612738  BOS  28  333   67  104   
195    201988   389        Patty Mills  1610612748  MIA  32  415   46  131   
196   1630249   386         Vit Krejci  1610612737  ATL  22  541   51  104   
210   1629007   397      Jontay Porter  1610612761  TOR  26  360   40  104   
212   1630551   414  Justin Champagnie  1610612764  WAS  15  235   32   78   
213   1630610   458      DeJon Jarreau  161061

In [ ]:
# Realizar o merge usando a coluna "PLAYER" como chave
box_df = pd.merge(box_df, dizzle_subset, on='PLAYER', how='left')

## Organizando dataset


### Adicionando uma coluna com a média dos rankings

In [ ]:
box_df['DIZZLE_RANK'] = box_df['DIZZLE_RANK'].astype(float)

In [ ]:
#Adicionando uma nova coluna chamada "Média de posição" ou "average" ou "avg_rank"
box_df['avg_rank'] = box_df[['BBM_RANK', 'FBI_RANK', 'DIZZLE_RANK']].mean(axis=1).round(2)

### Reorganizando ordem das colunas

In [ ]:
#Colocando avg_rank após coluna PLAYER
cols = list(box_df.columns)
cols.insert(3, cols.pop(cols.index('avg_rank')))
box_df = box_df[cols]

In [ ]:
#Colocando OWNER como primeira coluna
cols = list(box_df.columns)
cols.insert(0, cols.pop(cols.index('OWNER')))
box_df = box_df[cols]

In [ ]:
#Organizando tabela em ordem de time e jogadores com maior ranking
box_df = box_df.sort_values(by=['OWNER', 'avg_rank'])

In [ ]:
#Criando tabela com os top10 de cada time
df_top10 = df_sorted.groupby('OWNER').head(10)

In [ ]:
#Exportar times
df_top10.to_csv('/content/drive/MyDrive/NBA/ETL/box_10.csv', index=False)

In [ ]:
#Exportar times
box_df.to_csv('/content/drive/MyDrive/NBA/ETL/box_and_one_playoffs.csv', index=False)